In [69]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time
import sqlite3
import pickle
import matplotlib
import matplotlib.pyplot as plt
import sys
from requests.exceptions import ConnectionError

pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
matplotlib.rcParams['font.family'] = 'AppleGothic'

%matplotlib inline

#  Prep1. Create Country's List, Dictionary

In [2]:
r = requests.get("http://eikaiwa.dmm.com/list/")
bs_list = BeautifulSoup(r.text, "lxml")

countries = bs_list.find_all("span", {"class":"text_adjust"})

region_list = ["アジア","ヨーロッパ","北米","中南米","オセアニア","中東","アフリカ"]
region_dict = {}
country_list = []
native_countries = ['アイルランド','イギリス','南アフリカ','アメリカ合衆国','カナダ','オーストラリア','ニュージーランド']

for country in countries:
    country_list.append(country.getText().strip())

for i,country in enumerate(country_list):
    if country == "アジア":
        
        # when asia emerges twice (when the loop is in the 2nd iteration)
        if i > 1:
            break
        asia = i
    elif country == "ヨーロッパ" : euro = i
    elif country == "北米" : n_america = i
    elif country == "中南米" : s_america = i
    elif country == "オセアニア" : oceania = i
    elif country == "中東" : middle = i
    elif country == "アフリカ" : africa = i
    
region_dict["asia"] = country_list[asia+1: euro]
region_dict["europe"] = country_list[euro+1: n_america]
region_dict["north_america"] = country_list[n_america+1: s_america]
region_dict["south_america"] = country_list[s_america+1: oceania]
region_dict["oceania"] = country_list[oceania+1: middle]
region_dict["middle_east"] = country_list[middle+1: africa]
region_dict["africa"] = country_list[africa+1:]

In [3]:
region_dict["north_america"]

['アメリカ合衆国', 'カナダ']

# Prep2. Create tag list

ネイティブ、新人はtag内に入ってないので注意。
新人とはまだfavが付いてない講師を表しているため講師歴が３年以上でも新人で検索すると表示される講師もいる。
"Let's Go対応"に関してはスペースを取り除いた。

In [4]:
tags = bs_list.find("div",{"class":"special-checkbox"}).find_all("label")
tags_list = []

for tag in tags:
    tags_list.append(tag.getText().replace(" ", "").strip())

tags_list

['日本人講師',
 'ビジネス英会話',
 '講師歴3年以上',
 'キッズ向け',
 'TOEFL対応',
 '初心者向け',
 '上級者向け',
 'スピーキングテスト対応',
 "Let'sGo対応",
 '英検®対応',
 'IELTS',
 'ネイティブ講師',
 '新人講師']

# Prep3. Extract all teachers' URL (that is, all indices used for teachers' URL)
# (Don't Run!)

Begin scraping from the 2nd page of "all teachers".

In [9]:
teacher_url = []

In [ ]:
def append_teacher_url_to_list(url):
    bs = BeautifulSoup(requests.get(url).text, "lxml")
    teachers_in_page = bs.find_all("div",{"class":"profile_pictures non_favorite "})
    
    #get teacher indicies from the page
    for each_teacher in teachers_in_page:
        teacher_url.append("http://eikaiwa.dmm.com" + each_teacher.find("a")["href"])

In [ ]:
def get_next_page_url(url, page):
    bs = BeautifulSoup(requests.get(url).text, "lxml")
    urls = bs.find("div", {"class":"list-boxpagenation"}).find_all("a")

    for i in range(10):
        if urls[i].getText() == str(page + 1):
            prev_url = urls[i]["href"]
            return "http://eikaiwa.dmm.com" + prev_url
        
    exit()

In [ ]:
def get_previous_page_url(url):
    bs = BeautifulSoup(requests.get(url).text, "lxml")
    prev_url = bs.find("div", {"class":"list-boxpagenation"}).find_all("a")[0]["href"]
    return "http://eikaiwa.dmm.com" + prev_url

In [ ]:
# url of 2nd page of "all teachers"
second_url = "http://eikaiwa.dmm.com/list/?param=BFsBAVxNEggJVQsaQEIHRRU6Ql9dXEYCEVgDXBALWF4AQwtKXA5bEF0LVWdHXwtSQ15FDAQDRlcNDFJECRYNCF9DUlYJXQBQVAATA1oMVgwSXwIMEk0dSQdADAcIVQ1LGBIKDlwUAl1NC0VKShRdXltVDUUKD14bBQdZAlcXFQsMWwACFQxSCBoEVl0RDRUNV18U04mNjYTAQAwVCFENEgsAXVxEDS8JS18FAhFSB0MERw1FCghUA0BQB1cKSAYCSFIBG11FWwYCR0JXQUJEDBJfBwwSDUYCEVgOXBADRVUAPkdWFFJDCUtfAQIRFF1EW10MFFZYElYQC0MDEF55CxZbBgNEWABGURNUGgh4XURbVAcMElcFTQsUUjlBDVhHR1p*AhUMUggaC1RPEQ0PDVFeRQwBCF4bCBJoElcEVFgAE0MbXXhaQQJRCxpDVwFSQ15fDAICFwNRWBUSUwcVCwxbAQIb"

# url of 1st page of "all teachers"
first_url = get_previous_page_url(second_url)
append_teacher_url_to_list(first_url)

In [ ]:
current_url = second_url
scraped_pages = [1]

for i in range(500):
    try:
        current_page = i + 2
        append_teacher_url_to_list(current_url)
        current_url = get_next_page_url(current_url, current_page)
        scraped_pages.append(current_page)
        if current_page % 50 == 0 : print(current_page)
        time.sleep(1)
    except Exception as e:
        print(scraped_pages, len(teacher_url))
        print(current_page, e)
        break

# Store teachers' URL in a text file. (Don't Run)

In [ ]:
f = open('teacher_urls.txt', 'wb')
pickle.dump(teacher_url, f)
f.close()

# Load in teachers' URL to a list "urls" (Run!)

In [5]:
f = open('teacher_urls.txt', 'rb')
urls = pickle.load(f)
f.close()

In [6]:
len(urls)

4743

# Scrape raw teachers' info and store them into the database file

In [7]:
def get_evalu_counts(teacher_id):

    r =  requests.post("https://eikaiwa.dmm.com/teacher/tab_ajax_ratecomment/",
        data='teacher_id={}&page=1'.format(teacher_id),
        headers={
            "accept": "*/*",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "en-US,en;q=0.9",
            "authority": "eikaiwa.dmm.com",
            "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
            "origin": "https://eikaiwa.dmm.com",
            "referer": "https://eikaiwa.dmm.com/teacher/index/{}/".format(teacher_id),
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
            "x-requested-with": "XMLHttpRequest"
        },
        cookies={
            "AMP_TOKEN": "%24NOT_FOUND",
            "PHPSESSID": "d3b63b845f04033e8ba42225de244a88",
            "__utma": "225231877.1937935001.1545231753.1546318067.1547193623.3",
            "__utmb": "225231877.2.10.1547193623",
            "__utmc": "225231877",
            "__utmt": "1",
            "__utmz": "225231877.1547193623.3.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)",
            "_dc_gtm_UA-48257133-1": "1",
            "_dga": "GA1.3.1937935001.1545231753",
            "_dga_gid": "GA1.3.1694366917.1547193623",
            "_fbp": "fb.1.1547193626615.72004840",
            "_gali": "ratecomment",
            "app_uid": "ygb3J1waXYgdkRfaDtkSAg==",
            "cX_G": "cx%3A3kv82phj0hx3mw4ofrjh1f8ur%3A3cguwtwuxac6i",
            "cX_P": "jpvb385rfrdh4crx",
            "cX_S": "jqrr4xovsbqzvxt7",
            "i3_ab": "7460"
        },
    )
    bs = BeautifulSoup(r.text)
    n_5 = int(bs.find_all("span",{"class":"total_star"})[0].getText())
    n_4 = int(bs.find_all("span",{"class":"total_star"})[1].getText())
    n_3 = int(bs.find_all("span",{"class":"total_star"})[2].getText())
    n_2 = int(bs.find_all("span",{"class":"total_star"})[3].getText())
    n_1 = int(bs.find_all("span",{"class":"total_star"})[4].getText())
    
    #print(n_5, n_4, n_3, n_2, n_1)
    return [n_5, n_4, n_3, n_2, n_1]

In [8]:
def get_message(teacher_id):
    
    r = requests.post("https://eikaiwa.dmm.com/teacher/tab_ajax_message/",
        data='teacher_id={}'.format(teacher_id),
        headers={
            "accept": "*/*",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "ja,en-US;q=0.9,en;q=0.8",
            "authority": "eikaiwa.dmm.com",
            "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
            "origin": "https://eikaiwa.dmm.com",
            "referer": "https://eikaiwa.dmm.com//teacher/index/{}/area-message/".format(teacher_id),
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36",
            "x-requested-with": "XMLHttpRequest"
        },
        cookies={
            "AMP_TOKEN": "%24NOT_FOUND",
            "PHPSESSID": "fa0c214f4ff3c84e1f70d8607af7fe78",
            "__utma": "225231877.1937935001.1545231753.1545231753.1546318067.2",
            "__utmb": "225231877.1.10.1546318067",
            "__utmc": "225231877",
            "__utmt": "1",
            "__utmz": "225231877.1545231753.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)",
            "_dga": "GA1.3.1937935001.1545231753",
            "_dga_gid": "GA1.3.989924846.1546318069",
            "_fbp": "fb.1.1546318069039.130775676",
            "_gali": "message",
            "app_uid": "ygb3J1waXYgdkRfaDtkSAg==",
            "cX_G": "cx%3A3kv82phj0hx3mw4ofrjh1f8ur%3A3cguwtwuxac6i",
            "cX_P": "jpvb385rfrdh4crx",
            "cX_S": "jpvb385oxuzlj13w",
            "i3_ab": "7460"
        },
    )

    return r.text

In [14]:
def scrape_and_store_teacher_info_to_db(url_list, first_index, last_index):
    t = time.time()
    
    teachers = pd.DataFrame(columns=["id","name","country","age","gender","years","school",\
                                     "hobby","movie","tag","fav","evalu","n_evalu",\
                                     "rate5","rate4","rate3","rate2","rate1","message","staff_comment"])
    
    for i, url in enumerate(url_list[ first_index:last_index ]):

        try:
            # get url and nake beautiful soup object
            r = requests.get(url)
            bs = BeautifulSoup(r.text, 'lxml')

            # make the series to contain the info of teacher
            teacher = pd.Series(index=["id","name","country","age","gender","years","school",\
                                       "hobby","movie","tag","fav","evalu","n_evalu",\
                                       "rate5","rate4","rate3","rate2","rate1","message","staff_comment"])

            # store index number
            teacher["id"] = url.split("/")[-2]

            # scrape teacher name
            teacher["name"] = bs.find("div",{"class":"area-detail"}).find("h1").getText()


            # scrap characteristics

            info_types = bs.find("div", {"class":"confirm low"}).find_all("dt")
            info_values = bs.find("div", {"class":"confirm low"}).find_all("dd")

            for i_type, i_value in zip(info_types, info_values):
                if i_type.getText() == "国籍" : teacher.country = i_value.getText().strip()
                elif i_type.getText() == "年齢" : teacher.age = i_value.getText().strip()
                elif i_type.getText() == "性別" : teacher.gender = i_value.getText().strip()
                elif i_type.getText() == "経歴" : teacher.years = i_value.getText().strip()
                elif i_type.getText() == "出身校" : teacher.school = i_value.getText().strip()
                elif i_type.getText() == "趣味" : teacher.hobby = i_value.getText().strip()
                elif i_type.getText() == "好きな映画" : teacher.movie = i_value.getText().strip()
                elif i_type.getText() == "特徴" : teacher.tag = i_value.getText().strip()

            # Check if the teacher has evaluation score
            try:
                teacher.evalu = bs.find("li", {"id":"num"}).getText().strip()
                teacher.n_evalu = bs.find("p",{"class":"total","id":"total"}).getText()[1:-2]
                teacher[["rate5","rate4","rate3","rate2","rate1"]] = get_evalu_counts(int(teacher.id))
            except:
                print(i, "evaluation not found", url)

            # Check if the teacher has more than 0 favorite count
            try:
                teacher.fav = bs.find("span", {"id":"fav_count"}).getText().strip()
            except:
                print(i, "fav count not found". url)

            teacher.message = get_message(teacher.id)
            teacher.staff_comment = bs.find("div",{"class":"area-staffcomment"}).getText().strip()

            teachers = pd.concat([teachers, pd.DataFrame([teacher], columns=teacher.index)])

            if i % 200 == 0 : print(i)
            time.sleep(1)
        
        # Terinate without writing file if the connection fails in the middle
        except ConnectionError as e:    
            print(e)
            print("\nNo data was written in the database file.")
            print("Run this cell again")
            
            sys.exit(1)    
        
        except Exception as e:
            print(i,e,url)
            
    print(time.time() - t)
    
    # store teachers' info only when this part is reached to
    conn = sqlite3.connect("raw_teachers_data2.db")
    teachers.to_sql("profiles", conn, if_exists="append", index=False)
    print("All data was written in the database file")

In [15]:
scrape_and_store_teacher_info_to_db(urls, 0, 600)

0
152 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/27530/
200
272 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/24355/
400
558 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/25262/
575 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/26978/
3237.0670669078827
All data was written in the database file


In [16]:
scrape_and_store_teacher_info_to_db(urls, 600, 1200)

0
200
400
449 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/19373/
532 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/25866/
542 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/26432/
3253.2909440994263
All data was written in the database file


In [17]:
scrape_and_store_teacher_info_to_db(urls, 1200, 1800)

0
200
342 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/19927/
400
3266.6883261203766
All data was written in the database file


In [18]:
scrape_and_store_teacher_info_to_db(urls, 1800, 2400)

0
167 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/18783/
200
220 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/23728/
328 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/9925/
400
495 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/3811/
3251.4948449134827
All data was written in the database file


In [19]:
scrape_and_store_teacher_info_to_db(urls, 2400, 3000)

0
71 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/26822/
200
346 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/17481/
400
3204.755239009857
All data was written in the database file


In [20]:
scrape_and_store_teacher_info_to_db(urls, 3000, 3600)

0
125 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/26731/
200
310 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/17036/
334 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/22430/
400
545 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/15608/
577 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/23623/
3213.3905849456787
All data was written in the database file


In [21]:
scrape_and_store_teacher_info_to_db(urls, 3600, 4200)

0
200
241 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/22561/
381 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/26835/
400
404 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/11034/
515 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/18750/
540 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/6224/
548 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/15255/
593 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/14926/
3202.1952390670776
All data was written in the database file


In [22]:
scrape_and_store_teacher_info_to_db(urls, 4200, len(urls))

0
3 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/23841/
12 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/27241/
61 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/13777/
62 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/14138/
73 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/25799/
103 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/5687/
123 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/25339/
135 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/16608/
140 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/24673/
168 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/26177/
200 'NoneType' object has no attribute 'find' http://eikaiwa.dmm.com/teacher/index/12007/
289 'NoneType' 

# Load in row teacher info to DataFrame "teachers" from a database file

In [ ]:
# load teachers' data from database file to dataframe

conn = sqlite3.connect("raw_teachers_data2.db")
teachers = pd.read_sql("SELECT * FROM profiles",conn)
teachers.head()

In [135]:
teachers.shape, len(urls)

((4695, 20), 4743)

# Clean Teachers' DataFrame

In [154]:
teachers2 = teachers.copy()

In [155]:
teachers2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4695 entries, 0 to 4694
Data columns (total 20 columns):
id               4695 non-null float64
name             4695 non-null object
country          4695 non-null object
age              4597 non-null object
gender           4695 non-null object
years            4695 non-null object
school           4695 non-null object
hobby            4695 non-null object
movie            4695 non-null object
tag              4695 non-null object
fav              4695 non-null object
evalu            4652 non-null object
n_evalu          4652 non-null object
rate5            4652 non-null float64
rate4            4652 non-null float64
rate3            4652 non-null float64
rate2            4652 non-null float64
rate1            4652 non-null float64
message          4695 non-null object
staff_comment    4695 non-null object
dtypes: float64(6), object(14)
memory usage: 733.7+ KB


In [156]:
# confirm that there is no duplicate rows
assert teachers2.duplicated().sum() == 0

In [157]:
teachers2[teachers2.country == "日本"].staff_comment.unique()

array(['プラスネイティブプランのユーザー様に限り、日本人講師のレッスンをご予約いただけます（試験運用中）。「学習カウンセリング」を受けたり、英語に関する疑問を日本語で質問をすることが可能となります。「海外生活」、「旅行」、「試験対策」、など様々なトピックについてご相談いただけます。また、無料体験の際にも一度だけ受講可能です。「いきなり外国人の先生と話せない。」「レッスンの進め方が不安。」という方でも安心です。',
       'この講師は無料体験レッスン専用講師です。無料体験の際に一度だけ受講可能です。 「いきなり外国人の先生と話せない。」「レッスンの進め方が不安。」という方のためのプロの日本人講師です。\u3000日本人講師と一緒にレッスンの流れを掴んで素敵なオンライン英会話デビューを飾りましょう！※現在、試験的にネイティブプランのユーザーも受講可能としています。そちらは予告なく提供を終了する場合がございます。'],
      dtype=object)

In [158]:
#Convert columns to suitable types
teachers2.age = teachers2.age.fillna(-1)
teachers2[["id","age","fav"]] = teachers2[["id","age","fav"]].astype(int)
teachers2.evalu = teachers2.evalu.astype(float)
teachers2.age = teachers2.age.replace(-1,np.nan)

In [159]:
## make each tag's column from "tag"

# skip  "\n\n\n" between tags
teachers2.tag = teachers2.tag.replace("Let's Go対応","Let'sGo対応").replace(" ","").str.split()

In [160]:
# add columns of tags which is filled with 0

for tag in tags_list:
    teachers2[tag] = 0

In [161]:
# turn 0 into 1 in each column of tags according to the column "tag"

for i, t_list in enumerate(teachers2.tag):
    teacher_tags = set(t_list).intersection(set(tags_list))

    for each_tag in teacher_tags:
        teachers2.loc[teachers2.index[i], each_tag] = 1
        
    if (teachers.country.iloc[i] in native_countries):
        teachers2.loc[teachers2.index[i], "ネイティブ講師"] = 1
        
# drop column "tag" which is not needed any more
teachers2 = teachers2.drop(axis=1, labels="tag")

In [162]:
# remove " " in the column names (e.g.  "Let's Go対応 ")
teachers2.columns = teachers2.columns.str.replace(" ", "")

In [175]:
# convert the commma separated cell values into list of items

def make_list_of_items_in_a_column(df, col):
    
    list_of_items = []
    for each_cell in df[col]:
        items = []
        for each_item in each_cell.strip().replace("、",",").replace("。",",").split(","):
            if each_item not in  ["", "-", "."]:
                items.append(each_item)
        list_of_items.append(items)

    # dataframe which has a column with lists in it cannot be loaded into the sql (probably)
    # So that column has to be converted into string
    df[col] = list_of_items
    df[col] = df[col].astype(str)

In [164]:
make_list_of_items_in_a_column(teachers2, "hobby")
make_list_of_items_in_a_column(teachers2, "movie")

In [165]:
# make a column where regions of  teachers' country is shown

def country_to_region(country):
    for region, c_list in region_dict.items():
        if country in c_list:
            return region

teachers2["region"] = teachers2.country.apply(country_to_region)

In [166]:
# one hot encode "gender" as male :1, femal: 0
teachers2.gender = teachers2.gender.replace({"男性":1, "女性":0})
teachers2 = teachers2.rename(columns={"gender":"male"})

In [167]:
teachers2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4695 entries, 0 to 4694
Data columns (total 33 columns):
id               4695 non-null int64
name             4695 non-null object
country          4695 non-null object
age              4597 non-null float64
male             4695 non-null int64
years            4695 non-null object
school           4695 non-null object
hobby            4695 non-null object
movie            4695 non-null object
fav              4695 non-null int64
evalu            4652 non-null float64
n_evalu          4652 non-null object
rate5            4652 non-null float64
rate4            4652 non-null float64
rate3            4652 non-null float64
rate2            4652 non-null float64
rate1            4652 non-null float64
message          4695 non-null object
staff_comment    4695 non-null object
日本人講師            4695 non-null int64
ビジネス英会話          4695 non-null int64
講師歴3年以上          4695 non-null int64
キッズ向け            4695 non-null int64
TOEFL対応          46

In [168]:
# one hot encode "years" column and remove "講師歴3年以上", "新人講師" columns
one_hot_years = pd.get_dummies(teachers2.years)[["1年未満","1年","2年","3年以上"]].astype(int)

teachers2 = teachers2.join(one_hot_years)
teachers2 = teachers2.drop(axis=1, labels=["years","講師歴3年以上","新人講師"])

In [169]:
# convert staff_comment of Japanese teachers (プラスネイティブプランのユーザー様に限り.../ この講師は無料体験レッスン専用講師です...) into NaN

has_staff_comment = teachers2.staff_comment.str.contains("プラスネイティブプランのユーザー様に限り") | \
                        teachers2.staff_comment.str.contains("この講師は無料体験レッスン専用講師です")

teachers2.loc[has_staff_comment, "staff_comment"] = np.nan

In [170]:
teachers2.head()

,id,name,country,age,male,school,hobby,movie,fav,evalu,n_evalu,rate5,rate4,rate3,rate2,rate1,message,staff_comment,日本人講師,ビジネス英会話,キッズ向け,TOEFL対応,初心者向け,上級者向け,スピーキングテスト対応,Let'sGo対応,英検®対応,IELTS,ネイティブ講師,region,1年未満,1年,2年,3年以上
0,398,Jules（ジュルズ）,イギリス,39.0,0,South East Essex College,"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",4027,4.99,1745,1716.0,25.0,4.0,0.0,0.0,"My name is Jules and I am an English teacher from London, England. I have more than 5 years expe...",いつも生徒の声に耳を傾けてくれるジュルズ先生。イギリス・ロンドン出身のネイティブ講師で現在はタイで生活しています。これまで1年間ほど、タイで英語を教えた経験があり、タイ語で日常会話ができます。...,0,0,0,0,0,1,0,0,0,0,1,europe,0,0,0,1
1,1117,Sheri（シェリ）,フィリピン,33.0,0,University of Santo Tomas,"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",1347,5.00,376,373.0,3.0,0.0,0.0,0.0,I am Sheri and teaching is my passion. One of the most rewarding things in life for me is when a...,"""Teaching is my passion（教える事は情熱的なことだ）""と言い切るシェリ先生です。今までプライベート家庭教師として２年ほど韓国人相手に英語を教えた経験があります。子供相手に...",0,1,1,0,1,1,0,0,0,0,0,asia,0,0,0,1
2,1446,Ella（エラ）,フィリピン,36.0,0,CAISA (Career Institute Of Southeast Asia),"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",2802,5.00,4395,4145.0,188.0,49.0,8.0,5.0,hello everyone! My name is Shiella Mae. I prefer to be called Ella. I am from Davao City Philipp...,衛生コンサルタントの仕事を7年務めた経験を持つエラ先生。そのときの経験を活かして、情熱をもって英語を教えているそうです！時間があるときには、アメリカのドラマを観ていて、お気に入りの作品は、『ワ...,0,1,0,0,0,0,0,0,0,0,0,asia,0,0,0,1
3,1677,Dani（ダニ）,セルビア,32.0,0,"University of Belgrade, Serbia","[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",7404,5.00,2158,2068.0,76.0,9.0,2.0,3.0,Proverb says that a good teacher is like a candle - it consumes itself to light the way for othe...,5年間英語を教えてきたベテランのダニ先生。子供から大人、英語初級者から上級者まで幅広く指導してきました。現在の仕事では英語を多く用いるので、ビジネス英語を学びたい方にもおすすめです。また大の日...,0,0,0,0,1,0,0,0,0,0,0,europe,0,0,0,1
4,1897,Dina（ディナ）,セルビア,26.0,0,The College of Tourism,"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",3544,4.99,3283,3172.0,88.0,20.0,0.0,3.0,"I am glad to write something about myself :) My name is Dina, I'm 22 years old and I live and st...",ディナ先生の将来の夢は、ツアーアテンダントになることです。世界中を旅行して、たくさんの地域で人々を案内する仕事に就きたいと言います。これまでにセルビアの有名なホテルでの受付として働いたり、旅行...,0,0,0,0,1,0,0,0,0,0,0,europe,0,0,0,1


In [171]:
teachers2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4695 entries, 0 to 4694
Data columns (total 34 columns):
id               4695 non-null int64
name             4695 non-null object
country          4695 non-null object
age              4597 non-null float64
male             4695 non-null int64
school           4695 non-null object
hobby            4695 non-null object
movie            4695 non-null object
fav              4695 non-null int64
evalu            4652 non-null float64
n_evalu          4652 non-null object
rate5            4652 non-null float64
rate4            4652 non-null float64
rate3            4652 non-null float64
rate2            4652 non-null float64
rate1            4652 non-null float64
message          4695 non-null object
staff_comment    4609 non-null object
日本人講師            4695 non-null int64
ビジネス英会話          4695 non-null int64
キッズ向け            4695 non-null int64
TOEFL対応          4695 non-null int64
初心者向け            4695 non-null int64
上級者向け            469

# Store clean teachers' DataFrame into a database file

In [172]:
# store teahers' dataframe into database file "teachers_data.db"

conn = sqlite3.connect("teachers_data.db")
teachers2.to_sql("profiles", conn, if_exists="replace", index=False)

# Load in teachers' DataFrame to "df"

In [173]:
# load teachers' data from database file to dataframe

conn = sqlite3.connect("teachers_data.db")
df = pd.read_sql("SELECT * FROM profiles",conn)

In [174]:
df.head(5)

,id,name,country,age,male,school,hobby,movie,fav,evalu,n_evalu,rate5,rate4,rate3,rate2,rate1,message,staff_comment,日本人講師,ビジネス英会話,キッズ向け,TOEFL対応,初心者向け,上級者向け,スピーキングテスト対応,Let'sGo対応,英検®対応,IELTS,ネイティブ講師,region,1年未満,1年,2年,3年以上
0,398,Jules（ジュルズ）,イギリス,39.0,0,South East Essex College,"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",4027,4.99,1745,1716.0,25.0,4.0,0.0,0.0,"My name is Jules and I am an English teacher from London, England. I have more than 5 years expe...",いつも生徒の声に耳を傾けてくれるジュルズ先生。イギリス・ロンドン出身のネイティブ講師で現在はタイで生活しています。これまで1年間ほど、タイで英語を教えた経験があり、タイ語で日常会話ができます。...,0,0,0,0,0,1,0,0,0,0,1,europe,0,0,0,1
1,1117,Sheri（シェリ）,フィリピン,33.0,0,University of Santo Tomas,"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",1347,5.00,376,373.0,3.0,0.0,0.0,0.0,I am Sheri and teaching is my passion. One of the most rewarding things in life for me is when a...,"""Teaching is my passion（教える事は情熱的なことだ）""と言い切るシェリ先生です。今までプライベート家庭教師として２年ほど韓国人相手に英語を教えた経験があります。子供相手に...",0,1,1,0,1,1,0,0,0,0,0,asia,0,0,0,1
2,1446,Ella（エラ）,フィリピン,36.0,0,CAISA (Career Institute Of Southeast Asia),"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",2802,5.00,4395,4145.0,188.0,49.0,8.0,5.0,hello everyone! My name is Shiella Mae. I prefer to be called Ella. I am from Davao City Philipp...,衛生コンサルタントの仕事を7年務めた経験を持つエラ先生。そのときの経験を活かして、情熱をもって英語を教えているそうです！時間があるときには、アメリカのドラマを観ていて、お気に入りの作品は、『ワ...,0,1,0,0,0,0,0,0,0,0,0,asia,0,0,0,1
3,1677,Dani（ダニ）,セルビア,32.0,0,"University of Belgrade, Serbia","[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",7404,5.00,2158,2068.0,76.0,9.0,2.0,3.0,Proverb says that a good teacher is like a candle - it consumes itself to light the way for othe...,5年間英語を教えてきたベテランのダニ先生。子供から大人、英語初級者から上級者まで幅広く指導してきました。現在の仕事では英語を多く用いるので、ビジネス英語を学びたい方にもおすすめです。また大の日...,0,0,0,0,1,0,0,0,0,0,0,europe,0,0,0,1
4,1897,Dina（ディナ）,セルビア,26.0,0,The College of Tourism,"[['Reading', ' writing', ' art and jewelry design'], ['reading books', ' traveling', ' eating', ...","[['Bourne Trilogy'], ['Xmen Trilogy', ' Matrix Trilogy', ' A Beautiful Mind', ' Patch Adams', ' ...",3544,4.99,3283,3172.0,88.0,20.0,0.0,3.0,"I am glad to write something about myself :) My name is Dina, I'm 22 years old and I live and st...",ディナ先生の将来の夢は、ツアーアテンダントになることです。世界中を旅行して、たくさんの地域で人々を案内する仕事に就きたいと言います。これまでにセルビアの有名なホテルでの受付として働いたり、旅行...,0,0,0,0,1,0,0,0,0,0,0,europe,0,0,0,1
